In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('retina')

In [2]:
from __future__ import print_function, division
import os,sys
import numpy as np
import torch # pytorch package, allows using GPUs
# fix seed
seed=17
np.random.seed(seed)
torch.manual_seed(seed)
from datetime import datetime
from glob import glob
import pandas as pd
import h5py

# Step 1. Load Data

In [3]:
tmax_data = pd.read_csv('data/ghcnd_hcn_tmax.csv.gz', compression='gzip')
tmax_data.replace(-9999, np.nan, inplace=True)
#tmax_data.drop(columns=['latitude', 'longitude', 'elevation'], inplace=True)

# Step 2. Organize the Data

After discussing with Pankaj, here is how I'm going to assemble the data.  

- For each station, take a 3-D array (days x years x neighbor stations).  
- I'm going to go with 10 years of data for each sample and the 10 nearest stations.
    - Iterate through each week, choosing X to be the 10 previous years and y to be the subsequent week of temperatures for the station of interest.
- Start with using just the 20th century data, leaving all 21st century data for validation and testing later on. 

In [4]:
years = np.arange(1951, 2012, 1)

In [5]:
tmax_data_h20c = tmax_data[np.logical_and(tmax_data['year'] >= 1951, tmax_data['year'] <= 2011)].copy()

# remove any station that don't have at least 51 years of data in the range 1941 to 2011
tmp = []
counter = 0
for i, station in enumerate(tmax_data_h20c['station'].unique()):
    if (i+1)%10 == 0:
        print(i+1, '/', tmax_data_h20c['station'].nunique(), '\r', end='')
    a = tmax_data_h20c[tmax_data_h20c['station'] == station].copy()
    if a['year'].nunique() < 60:
        continue
    elif ((a['year'].nunique() >= 60) and (a['year'].nunique() < 61)):
        # add nans for the missing rows
        missing_years = years[~np.in1d(years, a['year'].to_numpy())]
        for missing_year in missing_years:
            a.loc[a.index[-1]+1] = np.hstack([station, missing_year, [np.nan]*365, a.iloc[0][['latitude', 'longitude', 'elevation']].copy().to_numpy()])
        a = a.sort_values('year')
        a.index = np.arange(len(a))
        tmp.append(a.copy().to_numpy())
        counter += 1
    elif a['year'].nunique() == 61:
        tmp.append(a.copy().to_numpy())
        counter += 1
    else:
        print('problem')
        break

tmax_data_h20c = pd.DataFrame(np.vstack(tmp), columns=tmax_data.columns)
del(tmp)

In [6]:
len(tmax_data_h20c)

10797

In [7]:
counter

177

In [8]:
counter * 61

10797

In [9]:
#station_info = pd.read_csv('data/ghcnd_hcn_tmax.csv.gz', compression='gzip')
station_info = tmax_data_h20c.copy()
#station_info = station_info[np.logical_and(station_info['year'] >= 1951, station_info['year'] <= 2001)]
station_info = station_info[['station', 'latitude', 'longitude', 'elevation']].copy()
station_info.drop_duplicates(inplace=True)

In [10]:
tmax_data_h20c.drop(columns=['latitude', 'longitude', 'elevation'], inplace=True)

In [11]:
def find_year_and_day(week, start_year=1961):
    ordered_day = week * 7
    year = start_year + ordered_day // 365
    day = ordered_day - (year - start_year)*365
    return year, day

In [12]:
def find_nearest_stations(station, nearest=10):
    global lat1, lon1, lat2, lon2, dist
    nearest = 10
    lat1, lon1 = station_info[station_info['station']==station].values[0][1:3]

    lat2 = station_info['latitude'].astype(float)
    lon2 = station_info['longitude'].astype(float)

    dist = np.arccos((np.sin(lat1 * np.pi/180) * np.sin(lat2 * np.pi/180) + \
                     np.cos(lat1 * np.pi/180) * np.cos(lat2 * np.pi/180) * np.cos((lon2 - lon1) * np.pi/180)).round(10)) * 6371
    close_stations = station_info.loc[dist.sort_values()[0:nearest].index, 'station']
    return close_stations#, dist.sort_values()[0:nearest]

In [13]:
weeks = np.arange(0, 365*50//7, 1)

In [145]:
X_train = []
y_train = []

for i, station in enumerate(tmax_data_h20c['station'].unique()):
    print(i+1, '/', len(tmax_data_h20c['station'].unique()), '\r', end='')
    for j, week in enumerate(weeks):
        if (j+1) % 100 == 0:
            print(j+1, '/', len(weeks), '\r', end='')
        nearest_stations = find_nearest_stations(station, nearest=10)
        near_data = tmax_data_h20c[np.in1d(tmax_data_h20c['station'], nearest_stations.to_numpy())]
        
        end_year, end_day = find_year_and_day(week)
        start_year = end_year - 10
        start_day = end_day
                
        end_year_y, end_day_y = find_year_and_day(week+1)
        
        time_data_X = near_data[np.logical_and(near_data['year'] >= start_year, near_data['year'] <= end_year)].copy()
        time_data_X.loc[time_data_X[time_data_X['year']==start_year].index, ['day' + str(day) for day in range(1, start_day+1)]] = np.nan
        time_data_X.loc[time_data_X[time_data_X['year']==end_year].index, ['day' + str(day) for day in range(end_day+1, 366)]] = np.nan
        
        if end_day > 357:
            time_data_y = near_data[np.logical_and(near_data['year'] >= end_year, near_data['year'] <= end_year_y)].copy() # 2 years for y that wraps
            time_data_y = time_data_y[time_data_y['station']==station] # only select the target station for y
            time_data_y1 = time_data_y.loc[:, ['day' + str(day) for day in range(end_day+1, 366)]] # first part of week
            time_data_y2 = time_data_y.loc[:, ['day' + str(day) for day in range(1, end_day_y+1)]] # first part of week
            time_data_y = np.hstack([time_data_y1.to_numpy(), time_data_y2.to_numpy()])
        else:
            time_data_y = near_data[near_data['year'] == end_year].copy() # 1 year for y that doesn't wrap
            time_data_y = time_data_y[time_data_y['station']==station] # only select the target station for y
            time_data_y = time_data_y.loc[:, ['day' + str(day) for day in range(end_day+1, end_day+8)]].to_numpy() # only select one week for y
        if len(time_data_y[0]) != 7:
            break
        
        stack_data = np.zeros((10, 11, 365))
        for k, near_station in enumerate(nearest_stations.to_numpy()):
            stack_data[k, :, :] = time_data_X[time_data_X['station']==near_station].loc[:, ['day' + str(day) for day in range(1, 366)]]
        
        X_train.append(stack_data)
        y_train.append(time_data_y[0])

In [146]:
info = []
counter = 0
for i, station in enumerate(tmax_data_h20c['station'].unique()):
    for j, week in enumerate(weeks):
        info.append([counter, station, find_year_and_day(week)[0], find_year_and_day(week)[1]])
        counter += 1

# Step 3.  Save all training data

In [147]:
np.savez_compressed('data/tmax_train/tmax_y_train.npz', np.array(y_train))

In [148]:
np.savez_compressed('data/tmax_train/tmax_X_train_info.npz', np.array(info))

In [149]:
len(X_train)//1000

461

In [150]:
i = 0
while i <= len(X_train)//1000:
    print(i+1, '/', 310, '\r', end='')
    with h5py.File('data/tmax_train/X_train_%.06d.h5'%(i*1000), 'w') as f:
        dset = f.create_dataset('data', data=np.nan_to_num(np.array(X_train[i*1000:i*1000+1000]), nan=-8888).astype(int), compression='gzip')
    i += 1

# Step 4. Organize validation data

In [14]:
tmax_data_val_raw = tmax_data[np.logical_and(tmax_data['year'] >= 2002, tmax_data['year'] <= 2016)].copy()
years = np.arange(2002, 2017, 1)

In [17]:
tmp = []
counter = 0
for i, station in enumerate(tmax_data_h20c['station'].unique()):
    if (i+1)%10 == 0:
        print(i+1, '/', tmax_data_h20c['station'].nunique(), '\r', end='')
    a = tmax_data_val_raw[tmax_data_val_raw['station'] == station].copy()
    if a['year'].nunique() < 15:
        # add nans for the missing rows
        missing_years = years[~np.in1d(years, a['year'].to_numpy())]
        for missing_year in missing_years:
            a.loc[a.index[-1]+1] = np.hstack([station, missing_year, [np.nan]*365, a.iloc[0][['latitude', 'longitude', 'elevation']].copy().to_numpy()])
        a = a.sort_values('year')
        a.index = np.arange(len(a))
        tmp.append(a.copy().to_numpy())
        counter += 1
    elif a['year'].nunique() == 15:
        tmp.append(a.copy().to_numpy())
        counter += 1
    else:
        print('problem')
        break

tmax_data_val = pd.DataFrame(np.vstack(tmp), columns=tmax_data.columns)
del(tmp)

In [18]:
tmax_data_val['station'].nunique()

177

In [21]:
weeks = np.arange(0, 365*5//7, 1)

X_val = []
y_val = []

for i, station in enumerate(tmax_data_val['station'].unique()):
    print(i+1, '/', len(tmax_data_val['station'].unique()), '\r', end='')
    for j, week in enumerate(weeks):
        if (j+1) % 100 == 0:
            print(j+1, '/', len(weeks), '\r', end='')
        nearest_stations = find_nearest_stations(station, nearest=10)
        near_data = tmax_data_val[np.in1d(tmax_data_val['station'], nearest_stations.to_numpy())]
        
        end_year, end_day = find_year_and_day(week, start_year=2012)
        start_year = end_year - 10
        start_day = end_day
                
        end_year_y, end_day_y = find_year_and_day(week+1, start_year=2012)
        
        time_data_X = near_data[np.logical_and(near_data['year'] >= start_year, near_data['year'] <= end_year)].copy()
        time_data_X.loc[time_data_X[time_data_X['year']==start_year].index, ['day' + str(day) for day in range(1, start_day+1)]] = np.nan
        time_data_X.loc[time_data_X[time_data_X['year']==end_year].index, ['day' + str(day) for day in range(end_day+1, 366)]] = np.nan
        
        if end_day > 357:
            time_data_y = near_data[np.logical_and(near_data['year'] >= end_year, near_data['year'] <= end_year_y)].copy() # 2 years for y that wraps
            time_data_y = time_data_y[time_data_y['station']==station] # only select the target station for y
            time_data_y1 = time_data_y.loc[:, ['day' + str(day) for day in range(end_day+1, 366)]] # first part of week
            time_data_y2 = time_data_y.loc[:, ['day' + str(day) for day in range(1, end_day_y+1)]] # first part of week
            time_data_y = np.hstack([time_data_y1.to_numpy(), time_data_y2.to_numpy()])
        else:
            time_data_y = near_data[near_data['year'] == end_year].copy() # 1 year for y that doesn't wrap
            time_data_y = time_data_y[time_data_y['station']==station] # only select the target station for y
            time_data_y = time_data_y.loc[:, ['day' + str(day) for day in range(end_day+1, end_day+8)]].to_numpy() # only select one week for y
        if len(time_data_y[0]) != 7:
            break
        
        stack_data = np.zeros((10, 11, 365))
        for k, near_station in enumerate(nearest_stations.to_numpy()):
            stack_data[k, :, :] = time_data_X[time_data_X['station']==near_station].loc[:, ['day' + str(day) for day in range(1, 366)]]
        
        X_val.append(stack_data)
        y_val.append(time_data_y[0])

In [23]:
len(X_val)

46020

# Step  5. Save validation data

In [24]:
val_info = []
counter = 0
for i, station in enumerate(tmax_data_val['station'].unique()):
    for j, week in enumerate(weeks):
        val_info.append([counter, station, find_year_and_day(week, start_year=2002)[0], find_year_and_day(week, start_year=2002)[1]])
        counter += 1

In [28]:
# Step 3.  Save all training data

np.savez_compressed('data/tmax_val/tmax_y_val.npz', np.array(y_val))

np.savez_compressed('data/tmax_val/tmax_X_val_info.npz', np.array(val_info))

In [26]:
len(X_val)//1000

46

In [29]:
i = 0
while i <= len(X_val)//1000:
    print(i+1, '/', len(X_val)//1000, '\r', end='')
    with h5py.File('data/tmax_val/X_val_%.06d.h5'%(i*1000), 'w') as f:
        dset = f.create_dataset('data', data=np.nan_to_num(np.array(X_val[i*1000:i*1000+1000]), nan=-8888).astype(int), compression='gzip')
    i += 1